MERLIN tutorial following http://www.black-hole.eu/index.php/schools-workshops-and-conferences/2nd-school-on-multiwavelength-astronomy/course-materials/164-merlin-data


## Overview

This tutorial describes the data reduction for MERLIN 5-GHz radio continuum observations of the radio galaxy 3C277.1. For more information see www.merlin.ac.uk.

These data were taken using 6 antennas, Ca (32 m) and Mk Ta Da Kn De, all 25 m. Their sensitivities vary, with De being by far the least efficient at this frequency. The array contains baselines up to 217 km and a usable bandwidth of 13 MHz which has been combined in one channel, hence the data volumne is small and rapid to process (the e-MERLIN upgrade will provide 2 GHz bandwidth). All antennas are alt-az. All four polarizations (RR RR LR RL) are present. Antenna positions are geocentric, terrestrial E, N positive.

The data can be downloaded from 3C277.1.MULTTB which is a multi-source file containing:

   - 3C277.1 (target)
   - 1300+580 (phase-reference source and polarization leakage calibrator)
   - 3C286 (primary flux scale calibrator and polarization angle calibrator)
   - OQ208 (point source flux scale calibrator)

Two flux scale calibrators are used because sources like 3C286 with stable, well-known amplitudes (and in this case polarization) are usually extended over several arcsec or more and hence are heavily resolved by MERLIN. On the other hand, point-like, bright sources like OQ208 are usually variable.

 
Information: If you want to short-circuit any of the steps, or compare your data (see if you can do better!), 3C277.1.FITS is a multi-source file containing target and calibration sources which has already been (mostly) edited. 3C277.1C.SPLIT contains the fully-edited target with all calibration from calibration sources applied, ready for imaging and self-calibration. 3C277.1.CAL.SPLIT is also fully self-calibrated and 3C277.1.FITSIMAGE is the final image in full polarization. These can be loaded into CASA using importuvfits for the first 3 files and importfits for the image.

The script below covers:

   - Loading and inspecting data
   - Editing
   - Initial phase-calibration of calibration sources
   - Setting the flux scale
   - Time-dependent amplitude calibration of calibration sources
   - Polarization leakage and angle calibration
   - Target imaging and self-calibration
   - Imaging in polarization
   - Image measurements 

In [ ]:
import os

In [ ]:
casa['build']['version']

## Import, inspect and flag the data

This assumes that all data are in the present working directory; if not, use the path in front of the fitsfile name.

Start CASA and set variables for convenience. Load the fits data using importuvfits and list the contents using listobs.  

In [ ]:
prefix = '3C277.1C'
msfile = prefix + '.ms'

In [ ]:
default(importuvfits)
importuvfits(fitsfile = "3C277.1.MULTTB",
    vis = msfile)

In [ ]:
default(listobs)
listobs(vis = msfile)

Plot amplitude as a function of uvdistance to check that the phase reference source is a point. 

In [ ]:
default(plotms)
plotms(vis = msfile, 
    xaxis = 'uvdist', 
    yaxis = 'amp',
    field = '1300+580', 
    correlation = 'RR,LL', 
    coloraxis = 'corr',
    plotfile = '1300+580_amp_uvdist.png',
    expformat = 'png'
    )

The plot shows some bad data but no systematic trend with uv distance.

Antenna '1' (Defford) is less sensitive than the others, so the data are noisier and we will flag by baseline (there are only 15). Inspect the phase reference source and flag the low points of amplitude and the sections where the phase cannot be followed. Use the Next button to iterate through the baselines. 

In [ ]:
plotms(vis = msfile,
    gridrows = 5,
    xaxis = 'time', 
    yaxis = 'amp',
    field = '1300+580', 
    correlation = 'RR,LL', 
    coloraxis = 'corr',
    iteraxis = 'baseline',
    plotfile = '1300+580_amp_time.png',
    expformat = 'png'
    )

In [ ]:
plotms(vis = msfile,
    gridrows = 5,
    xaxis = 'time', 
    yaxis = 'phase',
    field = '1300+580', 
    correlation = 'RR,LL', 
    coloraxis = 'corr',
    plotrange = [-1,-1,-180,180],
    iteraxis = 'baseline',
    plotfile = '1300+580_phase_uvdist.png',
    expformat = 'png')

The phase-reference source amplitudes and phases are shown above. The target is observed in the gaps between each scan.

If long periods of data are flagged then there will be no solutions derived for calibration on shorter intervals and the target data in these intervals will normally be flagged also when the calibration is applied.

Try a short cut on OQ208 to flag all the data when the source is very low (this takes a little longer, see progress in the logger). 

In [ ]:
plotms(vis = msfile,
    gridrows = 5,
    xaxis = 'time', 
    yaxis = 'phase',
    field = 'OQ208', 
    correlation = 'RR,LL', 
    coloraxis = 'corr',
    plotrange = [-1,-1,-180,180],
    iteraxis = 'baseline',
    extendflag = True,
    plotfile = 'OQ208_phase_uvdist.png',
    expformat = 'png')

Some OQ208 baselines after flagging.

3C286 is very resolved, but fortunately it does not seem to need any flagging. We will flag the target 3C277.1 after applying calibration. Back up with flagmanager - add a comment if you like! Use flagmanager any time later if you have done significant flagging. 

In [ ]:
default(flagmanager)
flagmanager(vis = msfile,
    mode = 'save',
    versionname = msfile+'_calsource.flags')

 Plot the antenna distribution using plotants to chose a reference antenna with good data with many short baselines (easier to calibrate the phase).  Ignore messages about not finding the position of MERLIN. I suggest '5' as the refant. 

In [ ]:
plotants(vis = msfile,
    figfile = prefix+'_plotants.png')
clearstat()

## Setting the flux scale

3C286 is a standard flux and polarization calibration source, with properties tabulated by Baars (1977). It is somewhat resolved even on the shortest MERLIN baselines at 5 GHz, hence a sub-standard flux density used (96% of Baars). It has 11.2% fractional polarization at uvPA 66 deg, which we use to work out Q and U and then insert the values in the MODEL data column of the measurement set using setjy: 

In [ ]:
ipol = 7.073
qpol = 0.112*ipol*cos(66*pi/180.0)
upol = 0.112*ipol*sin(66*pi/180.0)

default(setjy)
setjy(vis = msfile,
    field = '3C286',
    standard = 'manual',
    fluxdensity = [ipol,qpol,upol,0.0])

In order to get enough signal to noise for accurate amplitude solutions we need to average over a few minutes, but the phase changes more rapidly on the long MERLIN baselines. A lower signal to noise is adequate for phase solutions so we solve for phase first with a short solution interval (typically 0.5 min) before deriving amplitude corrections. Both types of calibration are derived using gaincal. 

In [ ]:
default(gaincal)
gaincal(vis = msfile,
    caltable = prefix+'_cals.phcal',
    field = '3C286,OQ208,1300+580',    # No spaces!
    solint = '0.5min',
    refant = '5',    # pick antenna with good data near array centre 
    calmode = 'p',
    minsnr = 3.0,
    minblperant = 3)

 Check the logger and plot the solutions: 

In [ ]:
default(plotcal)
plotcal(caltable = prefix+'_cals.phcal',
    field = '1300+580',
    subplot = 611,
    iteration = 'antenna',
    xaxis ='time',
    yaxis ='phase',
    plotrange = [-1, -1, -180, 180],
    figfile = prefix+'_cals.phcal.png')

Phase reference source phase solutions (the range is [-180,+180] deg)

Check the solutions for the other calibration sources. Even where the phase solutions are going round quite fast, they should not be random. 3C286 is heavily resolved on the longer baselines so we derive amplitude solutions for it and OQ208 on short baselines only, applying the phase solutions. minsnr=-1 is a fix to allow solutions to be passed on only 3 baselines. 

In [ ]:
gaincal(vis = msfile,
    caltable = prefix+'_cals.a1cal',
    field = '3C286,OQ208',    # No spaces!
    antenna = '4~6&4~6',
    solint = '2min',
    refant = '5',    # pick antenna with good data near array centre 
    calmode = 'ap',
    minsnr = -1,
    minblperant = 2,
    gaintable = prefix+'_cals.phcal')

 Check the solutions, which should be within a few percent of unity. 

In [ ]:
default(plotcal)
plotcal(caltable = prefix+'_cals.a1cal',
    field = '3C286',
    antenna = '4,5,6',
    subplot = 311,
    iteration='antenna',
    xaxis='time',
    yaxis='amp',
    plotrange=[-1, -1, -1, -1],
    figfile=prefix+'_3C286_cals.a1cal.png')

Also check OQ208 - this will have a different mean because there is not yet a proper model. Flag any solutions with a large scatter. 

In [ ]:
plotcal(caltable = prefix+'_cals.a1cal',
    field = 'OQ208',
    antenna = '4,5,6',
    subplot = 311,
    iteration='antenna',
    xaxis='time',
    yaxis='amp',
    plotrange=[-1, -1, -1, -1],
    figfile=prefix+'_OQ208_cals.a1cal.png')

fluxscale is used to compare the solutions for OQ208 with those of 3C286 to find the flux density of the former. 

In [ ]:
default(fluxscale)
fluxscale(vis = msfile,
    caltable = prefix+'_cals.a1cal',
    fluxtable = prefix+'_OQ208.flux',
    reference = '3C286',
    transfer = 'OQ208')

Note the OQ208 flux density in the logger; the scatter should be less than 1%. I got

Flux density for OQ208 in SpW=0 is: 2.3099 +/- 0.00530734 (SNR = 435.226, nAnt= 3) 

The fluxtable will contain the proper scaling, but since we have only used a subset of antennas we have to set the flux density of OQ208 (insert your value below) and repeat the amplitude calibration using all antennas:



In [ ]:
default(setjy)
setjy(vis = msfile,
    field = 'OQ208',
    standard = 'manual',
    fluxdensity = [2.31,0])

 Use this to find the flux density of the phase reference source on all baselines 

In [ ]:
gaincal(vis = msfile,
    caltable = prefix+'_cals.a2cal',
    field = 'OQ208,1300+580',
    antenna = '',
    solint = '2min',
    refant = '5',
    calmode = 'ap',
    minsnr = 3,
    minblperant = 4,
    gaintable = prefix+'_cals.phcal',
    selectdata = False)

 Use plotcal to check the solutions (see previously); assuming that they are OK use fluxscale: 

In [ ]:
plotcal(caltable = prefix+'_cals.a2cal',
    field = '1300+580',
    antenna = '4,5,6',
    subplot = 311,
    iteration='antenna',
    xaxis='time',
    yaxis='amp',
    plotrange=[-1, -1, -1, -1],
    figfile=prefix+'_1300+580_cals.a2cal.png')

In [ ]:
fluxscale(vis = msfile,
    caltable = prefix+'_cals.a2cal',
    fluxtable = prefix+'_1300+580.flux',
    reference = 'OQ208',
    transfer = '1300+580')

Note the flux density of 1300+580 from the logger and set this. This is not strictly necessary as you could simply use the rescaled fluxtable, but it makes accounting easier in later calibration. I got

Flux density for 1300+580 in SpW=0 is: 0.425989 +/- 0.000969224 (SNR = 439.515, nAnt= 6)

In [ ]:
setjy(vis = msfile,
    field = '1300+580',
    standard = 'manual',
    fluxdensity = [0.426,0])

## Phase reference source solutions

prefix+'_cals.phcal' (i.e. 3C277.1C_cals.phcal) contains phase-only solutions for all the calibration sources including the phase reference source. Make a calibration table for the phase reference source with the amplitude gains correctly scaled with respect to the derived flux density. 

In [ ]:
gaincal(vis = msfile,
    caltable = prefix+'_cals.acal',
    field = '1300+580',    # No spaces!
    antenna = '',
    solint = '2min',
    refant = '5',    # pick antenna with good data near array centre 
    calmode = 'ap',
    minsnr = 3,
    minblperant = 4,
    gaintable = prefix+'_cals.phcal',
    selectdata = False)

Check in plotcal - the amplitude gains should all be close to unity and the phase solutions should be small since we applied the initial phase calibration table. Note that, unlike AIPS, CASA uses the inverse of the gain correction factors. For example, the points which looked low on antenna 6 in the raw amplitudes, also have low amplitude gains correction factors. 

In [ ]:
plotcal(caltable = prefix+'_cals.acal',
    field = '1300+580',
    antenna = '',
    subplot = 611,
    iteration ='antenna',
    xaxis = 'time',
    yaxis = 'amp',
    plotrange = [-1, -1, 0.5, 1.5],
    figfile = prefix+'_1300+580_cals.acal.png')

## Polarization calibration

The phase reference source was observed over a wide hour angle range and it is point-like. We can use it to solve simultaneously for its intrinsic polarization and for leakage between the receiver feeds in orthogonal polarizations, using polcal. The parallactic angle correction (to allow for the rotation of alt-az feeds during tracking) is also derived. preavg defines the averaging intervals over which the parallactic angle correction is applied. 

In [ ]:
default(polcal)
polcal(vis = msfile,
    caltable = prefix+'_phref.dcal',
    field = '1300+580',
    selectdata = False,
    solint = 'inf',   
    combine = 'scan',
    preavg = 120.0,  
    refant = '5',    
    poltype = 'D+QU',
    minsnr = 3.0,
    minblperant = 3,
    gaintable = [prefix+'_cals.phcal', prefix+'_cals.acal'])

Inspect the solutions in the logger or use

```
caltable=prefix+'_phref.dcal'
listcal() 
```

The logger shows

polcal	Fractional polarization solution for 1300+580 (spw = 0): : Q = 0.0343205, U = -0.01451  (P = 0.0372617, X = -11.4588 deg)
...
polcal	  Ant=1: R: A=0.04847    P=-19.26   ; L: A=0.04227    P=-90.43  
polcal	  Ant=2: R: A=0.1035     P=-151.6   ; L: A=0.01853    P=-87.4   
polcal	  Ant=3: R: A=0.04337    P=-47.11   ; L: A=0.04684    P=-92.62  
polcal	  Ant=4: R: A=0.026      P=24.81    ; L: A=0.03879    P=-135.4  
polcal	  Ant=5: R: A=0          P=0        ; L: A=0.02193    P=-67.9   
polcal	  Ant=6: R: A=0.04473    P=-121.9   ; L: A=0.01694    P=51.1   

The source polarization is expected to be a few percent; I got P/I = 0.0373/0.426 (using the value of P above and the total flux density I from fluxscale). All amplitude (A) solutions should be less than 10% At present, the origin of phase, and hence the polarization angle, is arbitrary. This is corrected using the known polarization angle of 3C286. We apply the phase and polarization leakage solutions and use the model of its I Q U and V previously set. We rely on the amplitude being stable during the short (c. half an hour) observation, to avoid needing to use a model for amplitude calibration, since the polarization is dominated by the compact core.



In [ ]:
default(polcal)
polcal(vis = msfile,
    caltable = prefix+'_3C286.pacal',
    field = '3C286',
    selectdata = False,
    solint = 'inf',     # entire 'combine' intervals = scan length, here.
    combine = 'scan',
    preavg = 120.0,  # interval at which to apply ||actic angle
    refant = '5',    # pick antenna with good data near array centre           
    gaintable = [prefix+'_cals.phcal', prefix+'_phref.dcal'],
    poltype = 'Xf')

# The tutorial says poltype = 'X' but I get an error and it won't solve.
# Logger says Mean position angle offset solution for 3C286 (spw = 0) = -85.435 deg.

The polarization angle correction is reported in the logger.

polcal Position angle offset solution for 3C286 (spw = 0) = 33.9224 deg. 

## Apply the calibration and prepare for imaging

Apply the time-dependent phase and incremental phase plus amplitude solutions to the target and the phase-reference source using applycal. As the MERLIN antennas have different sensitivities we use calwt=False (so that data for the less sensitive antennas are not given too small a weight to be included properly in self-calibration). 

In [ ]:
default(applycal)
applycal(vis = msfile,
    gaintable = [prefix+'_cals.phcal', prefix+'_cals.acal', prefix+'_phref.dcal', prefix+'_3C286.pacal'],
    field = '1300+580,3C277.1',
    calwt = False,   
    parang = True)

 No amplitude calibration table for 3C286: 

In [ ]:
applycal(vis = msfile,
    gaintable = [prefix+'_cals.phcal',prefix+'_phref.dcal',prefix+'_3C286.pacal'],
    field = '3C286',
    calwt = False,   
    parang = True)

 This fills the CORRECTED data column with visibilities multiplied by the various calibration factors. split out the target (further imaging and self-calibration are quicker and simpler using a single-source file): 

In [ ]:
default(split)
split(vis = msfile,
    datacolumn = 'corrected',
    outputvis = '3C277.1C.split.ms',
    field = '3C277.1')

 Inspect the calibrated 3C277.1 data. 

In [ ]:
default(plotms)
plotms(vis = '3C277.1C.split.ms',
    gridrows = 5,
    xaxis = 'time',
    yaxis = 'amp',
    ydatacolumn = 'data',
    correlation = 'RR,LL',
    coloraxis = 'corr',
    iteraxis = 'baseline',
    field = '3C277.1',
    plotfile = '3C277.1.split_amp_time.png',
    expformat = 'png')

In [ ]:
plotms(vis = '3C277.1C.split.ms',
    gridrows = 5,
    xaxis = 'time',
    yaxis = 'phase',
    ydatacolumn = 'data',
    correlation = 'RR,LL',
    coloraxis = 'corr',
    iteraxis = 'baseline',
    plotrange = [-1,-1,-180,180],
    field = '3C277.1',
    plotfile = '3C277.1.split_phase_time.png',
    expformat = 'png')


Where there are no solutions from the phase reference, the data are already flagged. It is quite hard to see bad data on the long baselines so you might not be able to flag these but any obviously isolated points can be flagged. Inspect phase also. After later self-calibration, you might have another look and edit remaining isolated points, use the zoom if necessary.

## Self calibration and imaging of the target

Image the target using clean. The longest baseline is 217 km and the wavelength is 6 cm. Use this to estimate the interferometer resolution and hence a suitable pixel size to give at least 3 pixels across the beam. A minimum image size of at least 512 pixels is needed for for adequate deconvolution of the rather extended dirty beam produced by the sparse MERLIN array. psfmode='hogbom' provides a large beam patch for deconvolving extended sidelobes. Note that if interactive=True you have to set box(es) yourself and double click in them to activate. Don't clean too far before self-calibration. Ignor warnings about not finding position of MERLIN 

In [ ]:
default(clean)
clean(vis = '3C277.1C.split.ms',
    field = '3C277.1',
    imagename = '3C277.1C_clean1',
    cell = ['0.015arcsec', '0.015arcsec'],
    imsize = [512,512],
    psfmode = 'hogbom',
    imagermode = 'csclean',
    cyclefactor = 4,
    stokes = 'I',
#    interactive = True,
    niter = 1000,
    gain = 0.05,
    mask = '')

You can save your boxes for future use. In the viewer menu, select

```Tools > Region in File ```

Enter

```Stokes IQUV```

Then draw your boxes (you cannot save boxes drawn prior to openning the Region in File dialogue). Click on Save and enter a name, e.g. clean1.rgn. To save the clean inputs for future, use saveinputs

```saveinputs ('clean', '3C277.1.clean1.saved')```

Check progress in the logger. In this instance we have let CASA fit the natural synthesised beam, the size is reported in the logger (and also recorded in the image header). The Clean model is left in the MS in the MODEL_DATA column. Use the viewer to load 3C277.1C_clean1.image as a raster map and use the 'data display options' to tweak it. You can access these via the spanner icon or the Data menu option Ajust. 

In [ ]:
viewer('3C277.1C_clean1.image')

There are several ways to measure the noise and the peak. If you draw a box and double-click, the rms etc. will be displayed. To set the measurement in a variable (here called rms_target) use imstat, setting the limits to avoid the image. 

In [ ]:
default(imstat)
noise_target = imstat(imagename = '3C277.1C_clean1.image',
                box = '20,20,492,120',
                stokes = 'I')

rms_target = noise_target['rms'][0]

print rms_target
# 0.000896490093248

I got 0.00079 Jy. Use the menu Data>Load, select 3C277.1C_clean1.image as a contour image and use the 'data display options' to set the base contour level to the rms_noise and the relative contour levels to [-3,3,6,12,24,48,96]. 

First image of the target, contours at [-0.0025,0.0025, 0.005, 0.01, 0.02, 0.04, 0.08] Jy/beam

You can also overlay the Clean Components (3C277.1C_clean1.model) as a Marker Map using the Data > Open menu (set the x and y increments to 1.) Self-calibrate with gaincal using the model (the Fourier transform of the clean components) left in the MS. It is OK to selfcal in I if V is zero, which is usually the case at all but the highest resolution for extragalactic cm-wave radio continuum. 

In [ ]:
default(gaincal)
gaincal(vis = '3C277.1C.split.ms',
    caltable = prefix+'_self1.phcal',
    field = '3C277.1',
    refant = '5',
    minblperant = 3,
    minsnr = 1,
    calmode = 'p',
    solint = '2min',
    parang = True)

In [ ]:
plotcal(caltable=prefix+'_self1.phcal',
    xaxis='time',
    yaxis='phase',
    subplot=611,
    iteration='antenna',
    plotrange = [-1, -1, -180, 180],
    figfile = '3C277.1C_self1.phcal.png')

 You expect to see just residual solutions; if they look OK use applycal. 

In [ ]:
default(applycal)
applycal(vis = '3C277.1C.split.ms',
    gaintable = prefix+'_self1.phcal',
    field = '3C277.1',
    parang = True, 
    calwt = False)

Clean, set threshold to the measured noise, rms_target Restore the saved inputs (3C277.1.clean1.saved is just a text file, you can view or edit it) using the python command execfile. If you saved your boxes give the file name as the argument of mask. You can change these interactively in the clean viewer. Restore the previous, saved clean inputs 

In [ ]:
#execfile('3C277.1.clean1.saved')

Inspect and adapt the inputs, using the previous noise level as the minimum clean flux. You should stop cleaning when the minimum clean components have sidelobes fainter than the noise. You can calculate the stopping criteria more accurately by inspecting the sidelobe ratio of the dirty beam and comparing that with the expected final noise level. 

In [ ]:
clean(vis = '3C277.1C.split.ms',
    field = '3C277.1',
    imagename = '3C277.1C_clean2',
    cell = ['0.015arcsec', '0.015arcsec'],
    imsize = [512,512],
    psfmode = 'hogbom',
    imagermode = 'csclean',
    cyclefactor = 4,
    stokes = 'I',
#    interactive = True,
    niter = 2000,
    gain = 0.05,
    threshold=str(rms_target)+'Jy')
#    mask = 'clean1.rgn')

saveinputs ('clean', '3C277.1.clean2.saved')

 Make a new region mask if it needs improving. Inspect 3C277.1C_clean2.image as previously in the viewer and check that the noise has gone down. 

In [ ]:
viewer('3C277.1C_clean2.image')

In [ ]:
default(imstat)
noise_target = imstat(imagename='3C277.1C_clean2.image',
                    box='20,20,492,120',
                    stokes='I')

rms_target = noise_target['rms'][0]

print rms_target
# 0.000279023422363

In this example we go straight to amplitude self-calibration. If the previous phase solutions showed large changes you might do more phase-only calibration first to improve the model, perhaps trying shorter solution intervals and different boxes. Gaincal only works from the original, not corrected data so apply the previous phase self-cal solutions as the gaintable in gaincal. 

In [ ]:
default(gaincal)
gaincal(vis = '3C277.1C.split.ms',
    caltable = prefix+'_self2.ampcal',
    field = '3C277.1',
    refant = '5',
    minsnr = 1,
    calmode = 'ap',
    minblperant = 4,
    solint = '3min',
    gaintable = prefix+'_self1.phcal',
    parang =  True)

 Inspect the solutions. The amplitude corrections should be within a few percent of unity. Plot the phase corrections as well, these should be very small residuals. 

In [ ]:
plotcal(caltable=prefix+'_self2.ampcal',
    xaxis='time',
    yaxis='amp',
    subplot=611,
    iteration='antenna',
    plotrange = [-1, -1, 0.8, 1.2],
    figfile = '3C277.1C_self2.ampcal.png')

 Apply both 'p' and 'ap' calibration tables with applycal 

In [ ]:
applycal(vis = '3C277.1C.split.ms',
    gaintable = [prefix+'_self1.phcal', prefix+'_self2.ampcal'],
    field = '3C277.1',
    parang = True, 
    calwt = False)

 Finally, clean in full polarization. Remember that you can change the threshold or the number of iterations/cycles interactively. If you made a new file of saved clean boxes, update the name of the mask file. 

In [ ]:
# execfile('3C277.1.clean2.saved')

clean(vis = '3C277.1C.split.ms',
    field = '3C277.1',
    imagename = '3C277.1C_clean3',
    cell = ['0.015arcsec', '0.015arcsec'],
    imsize = [512,512],
    psfmode = 'hogbom',
    imagermode = 'csclean',
    cyclefactor = 4,
    stokes='IQUV',
#    interactive = True,
    niter = 5000,
    gain = 0.05,
    threshold=str(rms_target)+'Jy')
#    mask = 'clean1.rgn')

saveinputs ('clean', '3C277.1.clean3.saved')

You can use Display Panel options to view all 4 Stokes planes at once. In these settings, the mask is applied to all planes and imagermode='csclean' deconvolved them jointly, which is usually appropriate for small fractional polarization. View and check that the noise has gone down: 

In [ ]:
viewer('3C277.1C_clean3.image')

In [ ]:
default(imstat)
noise_target = imstat(imagename = '3C277.1C_clean3.image',
                box = '20,20,492,120',
                stokes = 'I')

rms_target=noise_target['rms'][0]

print rms_target
# 0.000105255082105

You can check the accuracy of your model by plotting the Fourier transform of the Clean components (in the MODEL data column of the MS) against the CORRECTED visibilities: 

In [ ]:
default(plotms)
plotms(vis = '3C277.1C.split.ms',
    xaxis = 'uvdist',
    yaxis = 'amp',
    ydatacolumn = 'corrected',
    selectdata = True,
    correlation = 'LL',
    field = '3C277.1',
    avgtime = '180', # 3 min, same as solution interval
    #plotcolor = 'blue',
    #overplot = True,
    plotfile = '3C277.1.split_FTcc.png',
    expformat = 'png')

In [ ]:
plotms(vis = '3C277.1C.split.ms',
    xaxis = 'uvdist',
    yaxis = 'amp',
    ydatacolumn = 'model',
    selectdata = True,
    correlation = 'LL',
    field = '3C277.1',
    avgtime = '180', # 3 min, same as solution interval
    #plotcolor = 'red',
    #overplot = True,
    #plotfile = '3C277.1.split_FTcc.png',
    expformat = 'png')
    
# This was originally done in plotxy but doesn't look like it works in plotms

Target model (red) overplotted on the calibrated visibility data.

Some scatter in the data is due to real structure below the CC cutoff and to unavoidable noise, which is worse on intermediate baselines because these include Defford, the least sensitive antenna at 5 GHz. If the model is a very poor fit, try more editing and/or self-calibration and/or change the clean inputs Split the total intensity Stokes I image out of final image using immath

In [ ]:
default(immath)
immath(imagename = '3C277.1C_clean3.image',
    mode = 'evalexpr',
    stokes = 'I',
    outfile = '3C277.1C.icln',
    expr = 'IM0')

 Use the toolkit to make a Q+iU image (needed to plot scaled polarization vectors) 

In [ ]:
# maybe skip this

#potool = casac.homefinder.find_home_by_name('imagepolHome')
#pho = potool.create()
#pho.open('3C277.1C_clean3.image')
#complexlinpolimage = '3C277.1C.cmplxlinpol'
#pho.complexlinpol(complexlinpolimage)
#pho.close()

 Estimate a suitable cutoff for plotting polarization e.g. 

In [ ]:
#print 10*rms_target

and then use the menu Data>Load to enter

'3C277.1C.cmplxlinpol'['3C277.1C.icln'> 0.0024] 

in the Load Data LEL expression field, as shown below (change 0.0024 to your value). (DON'T click on anything in the Name list). Click on Vector Map. 

 How to display scaled polarization vectors from the Q+iU image with a cutoff at about 10-20 times the rms of the Stokes I image.

Use Data Display options for the raster and vector images and try adding contours. Compare this with the image made before self-calibration - note that the 3 rms_noise level used as the contour base is now much lower. 

This plot uses x- and y-increments of 2, debiasing true, amplitude scale factor 2, vector color cyan and contours at: [-0.0004 0.0004 0.0008 0.0016 0.0032 0.0064 0.0128] Jy/beam

If time, try to improve the image... More self-calibration, maybe editing, clean more slowly with smaller gain, careful boxing... You could also split out 3C286 and 1300+580 and image these in full polarization as above. There is probably no need for further self calibration of the calibration sources.
Anita Richards 19 June 2010 